In [2]:
!pip install fuzzywuzzy

In [8]:
import pandas as pd
from fuzzywuzzy import process

# Step 1: Load the datasets
aid_data = pd.read_csv("aiddata_grant_loan.csv", sep=';')
outcome_data = pd.read_excel("OutcomeData (1).xlsx")

# Step 2: Match French recipient country names to English names
aid_countries = aid_data['recepient'].unique()
outcome_countries = outcome_data['Country_Name'].unique()

country_map = {}
for fr_country in aid_countries:
    match, score = process.extractOne(fr_country, outcome_countries)
    country_map[fr_country] = match if score > 80 else None

# Manual corrections for unmatched countries
manual_country_fixes = {
    'Erythree': 'Eritrea',
    'Soudan du Sud': 'South Sudan',
    'Maurice': 'Mauritius',
    'Gambie': 'Gambia',
    'Afrique centrale non specifie': 'Central Africa (unspecified)',
    'Republique centrafricaine': 'Central African Republic',
    'Afrique du Sud': 'South Africa',
    'Djibouti': 'Djibouti',
    'Afrique occidentale non specifie': 'Western Africa (unspecified)',
    'Albanie': 'Albania',
    'Afrique australe non specifie': 'Southern Africa (unspecified)',
    'Afrique orientale non specifie': 'Eastern Africa (unspecified)',
    'Sainte-Helene': 'Saint Helena',
    'Mayotte': 'Mayotte'
}
country_map.update(manual_country_fixes)

# Apply mapping
aid_data['recepient_en'] = aid_data['recepient'].map(country_map)

# Step 3: Transform outcome data to wide format
outcome_melted = outcome_data.melt(
    id_vars=['Country_Name', 'Series_Name'],
    var_name='year',
    value_name='value'
)

# Clean year column (extract just the 4-digit year)
outcome_melted['year'] = pd.to_numeric(outcome_melted['year'].str.extract(r'(\d{4})')[0], errors='coerce')

# Pivot so each series is a separate column
outcome_pivot = outcome_melted.pivot_table(
    index=['Country_Name', 'year'],
    columns='Series_Name',
    values='value',
    aggfunc='first'
).reset_index()

# Step 4: Merge Aid and Outcome data
merged = pd.merge(
    outcome_pivot,
    aid_data,
    how='left',
    left_on=['Country_Name', 'year'],
    right_on=['recepient_en', 'year']
)

# Step 5: Add binary column for aid presence
merged['aid_received'] = merged['aidamount'].notna().astype(int)

# Step 6: Final selection of relevant columns
final_df = merged[[
    'Country_Name', 'year', 'donar', 'measure', 'aidamount', 'aid_received'
] + [col for col in merged.columns if col not in [
    'Country_Name', 'year', 'recepient', 'recepient_en',
    'donar', 'measure', 'aidamount', 'aid_received',
    'unitmeas', 'Type de prix', 'baseperiod', 'unitmult',
    "Multiplicateur d'unite", 'flowtype', 'Type de flux'
]]]

# Replace French measure names with standard English labels
measure_translation = {
    "Aide alimentaire": "Development Food Aid",
    "Aide alimentaire developpementale": "Development Food Aid",
    "Aide humanitaire": "Humanitarian Aid",
    "Aide humanitaire": "Humanitarian Aid",
    "APD en % du RNB du receveur": "ODA as % of GNI of Recipient",
    "APD par habitant": "ODA per Capita",
    "Cooperation technique,": "Technical Cooperation",
    "Coopération technique": "Technical Cooperation",
    "Dons d'APD, versements": "ODA Grants, Disbursements",
    "Prets d'APD,": "ODA Loans"
}

final_df['measure'] = final_df['measure'].replace(measure_translation)

# Optional: Save to CSV
final_df.to_csv("merged_aid_loan_grant_outcome_data.csv", index=False)

/tmp/ipython-input-8-3301236761.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['measure'] = final_df['measure'].replace(measure_translation)


In [11]:
final_df.measure.unique()

array(['Development Food Aid', 'Humanitarian Aid',
       'Cooperation technique, versements', 'ODA Grants, Disbursements',
       "Prets d'APD, versements net", nan], dtype=object)